In [141]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.metrics import mean_squared_error,recall_score,f1_score,accuracy_score,auc
from scipy.stats import zscore

In [142]:
df=pd.read_csv('datasets\Health Insurance.csv')
df.head().T

,0,1,2,3,4
age,19,18,28,33,32
sex,female,male,male,male,male
bmi,27.9,33.77,33,22.705,28.88
children,0.0,1.0,3.0,0.0,0.0
smoker,yes,no,no,no,no
region,southwest,southeast,southeast,northwest,northwest
charges,16884.924$,1725.5523$,4449.462$,21984.47061$,3866.8552$


In [143]:
df=df.drop_duplicates()

In [144]:

noise=[]
for i in df.age:
    try:
        float(i)
    except:
        noise.append(i)
print(*noise)

/* " "


In [145]:
df.age = pd.to_numeric(df.age,errors="coerce")

In [146]:
df[df['age'].isnull()]

,age,sex,bmi,children,smoker,region,charges
408,NaN,male,21.12,3.0,no,southeast,6652.5288$
429,NaN,female,30.4,3.0,no,northwest,18804.7524$
471,NaN,female,30.115,0.0,no,northeast,2203.47185$
501,NaN,male,26.03,0.0,no,northeast,6837.3687$
531,NaN,female,31.73,0.0,no,northeast,14043.4767$
549,NaN,female,46.2,0.0,yes,southeast,45863.205$
572,NaN,female,43.12,2.0,no,southeast,4753.6368$
1320,NaN,male,31.065,3.0,no,northwest,5425.02335$
1554,NaN,female,31.35,NaN,yes,southeast,5469.0066$


In [147]:
df.bmi=pd.to_numeric(df.bmi,errors="coerce")

In [148]:
df[(df.sex=='female')&(df.bmi>30)&(df.bmi<50)]['age'].mode()

0    18.0
Name: age, dtype: float64

In [149]:
df.loc[(df.age.isnull()) & (df.sex == 'female'), 'age'] = 18

In [150]:
df[(df.sex=='male')&(df.bmi>20)&((df.bmi<32))]['age'].mode()

0    19.0
Name: age, dtype: float64

In [151]:
df.loc[(df.age.isnull()) & (df.sex == 'male'), 'age'] = 19

In [152]:
df.age.unique()

array([19., 18., 28., 33., 32., 31., 46., 37., 60., 25., 62., 23., 56.,
       27., 52., 30., 34., 59., 63., 55., 22., 26., 35., 24., 41., 38.,
       36., 21., 48., 40., 58., 53., 43., 64., 20., 61., 44., 57., 29.,
       45., 54., 49., 47., 51., 42., 50., 39.])

In [153]:
df[(df.age>=18)&(df.age<50)&(df.sex=='male')]['bmi'].median()

30.2

In [154]:
df.loc[(df['bmi'].isnull())&(df.sex=='male'),'bmi']=30.2

In [155]:
df[(df.age>=30)&(df.age<55)&(df.sex=='female')]['bmi'].median()

29.9

In [156]:
df.loc[(df['bmi'].isnull())&(df.sex=='female'),'bmi']=29.9

In [157]:
#BMI
df[(df.bmi.isnull())&(df.sex=='female')]

,age,sex,bmi,children,smoker,region,charges


In [158]:
df.loc[df.children.isnull(),'children']=0

In [159]:
df.loc[df.region.isnull(),'region']='unknown'

In [160]:
df.charges=df.charges.str.replace("$","",regex=False)

In [161]:

noise=[]
for i in df.charges:
    try:
        float(i)
    except:
        noise.append(i)
print(*noise)

- -  ' - -  - -


In [162]:
df.charges = pd.to_numeric(df.charges,errors="coerce")

In [163]:
df[df.charges.isnull()]

,age,sex,bmi,children,smoker,region,charges
232,19.0,female,17.800,0.0,no,southwest,NaN
316,50.0,male,32.205,0.0,no,northwest,NaN
409,32.0,male,30.030,1.0,no,southeast,NaN
411,44.0,female,20.235,1.0,yes,northeast,NaN
492,18.0,female,25.080,0.0,no,northeast,NaN
529,18.0,male,25.460,0.0,no,northeast,NaN
578,52.0,male,30.200,1.0,no,southwest,NaN
1328,23.0,female,24.225,2.0,no,northeast,NaN


In [164]:
def even(value):
    if value%2==0:
        return "yes"
    else:
        return "no"

In [165]:
df['P-BMI']=df.bmi.apply(even)

In [167]:
scaler=StandardScaler()
df[['age','bmi','children','charges']]=scaler.fit_transform(df[['age','bmi','children','charges']])
encoder = LabelEncoder()
df[['age','sex','smoker','region','P-BMI']]=df[['age','sex','smoker','region','P-BMI']].apply(encoder.fit_transform)

In [168]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,1553,1554,1555,1556,1557,1558,1559,1560,1561,1562
age,1.000000,0.000000,10.000000,15.000000,14.000000,13.000000,28.000000,19.000000,19.000000,42.000000,...,6.000000,0.000000,41.000000,18.000000,11.000000,24.000000,30.000000,21.000000,45.000000,36.000000
sex,1.000000,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,...,2.000000,1.000000,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000
bmi,-0.446432,0.509933,0.384482,-1.292824,-0.286767,-0.798349,0.456168,-0.472500,-0.131989,-0.782056,...,1.656920,0.115656,0.889547,-1.215435,-2.157138,-1.684657,0.759207,-1.029701,-1.977921,-0.689190
children,-0.904979,-0.071502,1.595452,-0.904979,-0.904979,-0.904979,-0.071502,1.595452,0.761975,-0.904979,...,-0.904979,-0.904979,-0.904979,-0.904979,-0.904979,-0.904979,-0.904979,-0.904979,-0.904979,-0.904979
smoker,3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,2.000000,3.000000,2.000000,3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
region,4.000000,3.000000,3.000000,2.000000,2.000000,3.000000,3.000000,2.000000,1.000000,2.000000,...,4.000000,3.000000,3.000000,3.000000,4.000000,4.000000,4.000000,3.000000,4.000000,1.000000
charges,0.299136,-0.952151,-0.727313,0.720063,-0.775403,-0.784502,-0.414386,-0.493551,-0.565783,1.292795,...,-0.589982,-0.643158,-0.951987,-0.260310,-0.408587,-0.930780,-0.891734,-0.091945,-0.304902,-0.012239
P-BMI,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [169]:
desc=pd.DataFrame(index=list(df))
desc['count']=df.shape[0]
desc['null']=df.isnull().sum()
desc['dtype']=df.dtypes
desc['unique']= df.nunique()
desc['%unique'] = desc['unique']/len(df)
desc=pd.concat([desc,df.describe().T],axis=1)
desc

,count,null,dtype,unique,%unique,count,mean,std,min,25%,50%,75%,max
age,1562,0,int64,47,0.030090,1562.0,2.100064e+01,14.185894,0.000000,8.000000,21.000000,33.000000,46.000000
sex,1562,0,int32,3,0.001921,1562.0,1.507042e+00,0.501390,0.000000,1.000000,2.000000,2.000000,2.000000
bmi,1562,0,float64,547,0.350192,1562.0,-4.867354e-16,1.000320,-4.992021,-0.704667,-0.050119,0.676523,3.664148
children,1562,0,float64,6,0.003841,1562.0,-1.990157e-17,1.000320,-0.904979,-0.904979,-0.071502,0.761975,3.262406
smoker,1562,0,int32,4,0.002561,1562.0,2.205506e+00,0.410491,0.000000,2.000000,2.000000,2.000000,3.000000
region,1562,0,int32,6,0.003841,1562.0,2.532010e+00,1.127065,0.000000,2.000000,3.000000,4.000000,5.000000
charges,1562,8,float64,1329,0.850832,1554.0,1.508875e-16,1.000322,-1.001980,-0.700904,-0.327841,0.263608,4.169164
P-BMI,1562,0,int32,2,0.001280,1562.0,1.152369e-02,0.106762,0.000000,0.000000,0.000000,0.000000,1.000000
